In [ ]:
import illustris_python as il
import matplotlib.pyplot as plt
from tenet.util import sphMap
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.patheffects as pe
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter, fourier_gaussian
from scipy.stats import ks_2samp, anderson_ksamp
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy import interpolate
import os
import time
import h5py
import rohr_utils as ru 
import random
import six
%matplotlib inline 

plt.style.use('fullpage.mplstyle')
figsizewidth  = 6.902 # the textwidth in inches of MNRAS


In [ ]:
def model_2dradprof(data):
    """ 
    compute the model 2D radial profile from a 2D histogram
    based on StackOverflow answer found here: 
    https://stackoverflow.com/questions/21242011/most-efficient-way-to-calculate-radial-profile
    """
    center = [(data.shape[0] - 1.) / 2., (data.shape[1] - 1.) / 2.]
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    
    rpix = np.arange(radialprofile.size)
    radprof_func = interpolate.interp1d(rpix, radialprofile)
    
    model = radprof_func(r)
    
    return model, radprof_func 


In [ ]:
indirec = '../Output/L680n8192TNG_subfindGRP/'
infname = 'central_subfind_L680n8192TNG_tau_all.hdf5'
centrals_taudict = {}
with h5py.File(indirec + infname, 'r') as f:
    group = f['Group']
    for dset_key in group.keys():
        centrals_taudict[dset_key] = group[dset_key][:]
    f.close()
    
haloIDs = centrals_taudict['HostSubhaloGrNr']


In [ ]:
indirec = '../Output/L680n8192TNG_subfindGRP/'
infname = 'subfind_L680n8192TNG_tau_all_gasz0.hdf5'
satellites_taudict = {}
with h5py.File(indirec + infname, 'r') as f:
    group = f['Group']
    for dset_key in group.keys():
        satellites_taudict[dset_key] = group[dset_key][:]
    f.close()

HostSubhaloGrNr = satellites_taudict['HostSubhaloGrNr']
subfindIDs = satellites_taudict['SubfindID']

In [ ]:

# load halo's gas cells
sim = 'L680n8192TNG'
basePath = ru.ret_basePath(sim)
snapNum = 99

# load general simulation parameters
header = il.groupcat.loadHeader(basePath, snapNum)
z = header['Redshift']
a = header['Time'] # scale factor
h = header['HubbleParam'] # = 0.6774
boxsize = header['BoxSize'] * a / h
    
gas_ptn = il.util.partTypeNum('gas')

xray_key = 'xray_lum_0.5-2.0kev'

gas_hsml_fact = 1.5
pixel_size = 5. # kpc
smoothing_scale = 300. / pixel_size # keep constant at 200 or 500 kpc
contours_smoothing = 7.5 / pixel_size
boxSizeSim = [boxsize, boxsize, boxsize]
ndims = 3

# open the centrals GRP file, loop over the keys, but only add the dataset for z=0



In [ ]:
# load all maps into dictionary 
indirec = '../Output/%s_subfindGRP/'%sim
infname = 'central_subfind_L680n8192TNG_branches.hdf5'
Lx_maps = {}
with h5py.File(indirec + infname, 'r') as inf:
    for group_key in inf.keys():
        group = inf[group_key]
        Lx_maps[group_key] = {}
        for dset_key in group.keys():
            if 'xray' in dset_key:
                Lx_maps[group_key][dset_key] = group[dset_key][:]
    inf.close()            
central_keys = np.array(list(Lx_maps.keys()))

In [ ]:
# load all subhalos in the simulation
subhalo_fields = ['SubhaloMassInRadType', 'SubhaloPos', 'SubhaloMassType']
star_ptn = il.util.partTypeNum('star')

Subhalos = il.groupcat.loadSubhalos(basePath, snapNum, fields=subhalo_fields)
Subhalos_Mstar = subhalos['SubhaloMassInRadType'][:,star_ptn] * 1.0e10 / h
Subhalos_Pos = subhalos['SubhaloPos'] * a / h
Subhalos_Mgas = subhalos['SubhaloMassType'][:,gas_ptn] * 1.0e10 / h

In [ ]:
group_index = 0
haloID = haloIDs[group_index]
group = Lx_maps[central_keys[group_index]]

proj = 'xy'
axes = [0,1]
FoF_Lxsoft = group['xray_lum_0.5-2.0kev_%s'%proj]
FoF_Lxsoft_smooth = group['xray_lum_0.5-2.0kev_%s_smooth'%proj]
FoF_Lxsoft_excess = FoF_Lxsoft / FoF_Lxsoft_smooth

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
M200c = halo['Group_M_Crit200'] * 1.0e10 / h
R200c = halo['Group_R_Crit200'] * a / h
halo_pos = halo['GroupPos'] * a / h
central = il.groupcat.loadSingle(basePath, snapNum, subhaloID=halo['GroupFirstSub'])
central_Mstar = central['SubhaloMassInRadType'][star_ptn] * 1.0e10 / h



In [ ]:
# conditions for another galaxy to be considered:
# i) M* > 10^10 and M_star^sat / M_star^cent < 1/10
# ii) projected host centric distance < 1.5 R200c

Mstar_lolim = 1.0e10 # Msun
Mstar_hilim_frac = 1./10. # M_star^cent
subhalo_indices = ((Subhalos_Mstar > Mstar_lolim) & 
                   ((Subhalos_Mstar / central_Mstar) < Mstar_hilim_frac) &
                   (np.sqrt((Subhalos_Pos[:,axes[0]] - halo_pos[axes[0]])**2 +
                            (Subhalos_Pos[:,axes[1]] - halo_pos[axes[1]])**2) < 1.5 * R200c))
subhaloIDs = np.where(subhalo_indices)[0]

subhalos_mstar = Subhalos_Mstar[subhalo_indices]
subhalos_mgas = Subhalos_Mgas[subhalo_indices]
subhalos_hcd_phys = np.sqrt((Subhalos_Pos[subhalo_indices,axes[0]] - halo_pos[axes[0]])**2 +
                           (Subhalos_Pos[subhalo_indices,axes[1]] - halo_pos[axes[1]])**2)
subhalos_hcd_norm = subhalos_hcd_phys / R200c
subhalos_m200c = np.ones(subhaloIDs.size, M200c.dtype) * M200c


In [ ]:
size = 3.0e2 # kpc
pixel_size = 5.0 # kpc
x = np.logspace(np.log10(1.), np.log10(size / pixel_size))
xs = np.tile(x, (subhaloIDs.size,1)) * pixel_size
ys = np.zeros((subhaloIDs.size, x.size), dtype=x.dtype) - 1
for subhalo_index, subhaloID in enumerate(subhaloIDs):
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
    SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

    center = [(FoF_Lxsoft_excess.shape[0] - 1.) / 2., (FoF_Lxsoft_excess.shape[1] - 1.) / 2.]
    extent_indices = [int(center[0] + (SubhaloPos[axes][1] - size)/pixel_size), int(center[0] + (SubhaloPos[axes][1] + size)/pixel_size),
                      int(center[1] + (SubhaloPos[axes][0] - size)/pixel_size), int(center[1] + (SubhaloPos[axes][0] + size)/pixel_size)]
    if min(extent_indices) < 0:
        print('satellite on the edge of the FoV: subhalo_index %d, subhaloID %d'%(subhalo_index, subhaloID))
        continue
    vals = FoF_Lxsoft_excess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]]
    model, func = model_2dradprof(vals)
    ys[subhalo_index,:] = func(x)
    
ys = np.ma.masked_values(ys, -1)

In [ ]:
def return_stacked_dict(ys, dset, dset_bins):
    """
    Given the tau_dict, dset_key, and dset_dset_bins, bin all subhalos within
    the tau_dict by the dset_key and dset_dset_bins. Then within the stack,
    calculate the medians, 16th, and 84th percentiles of the densities both
    in normalized and physical radial coordinates. Note that this assumes linear
    interpolation between the nonzero density entries. 
    Resulting dictionary takes indices as keys, which lead to another dictionary 
    that takes '50_phys', '16_phys', '84_phys', '50_norm', '16_norm', '84_norm',
    and 'Ngal'. 'Ngal' is an integer, and all other entries are 1D arrays 
    corresponding to the global radii_phys_bincents and radii_norm_bincents 1D arrays.
    """
    
    result_dict = {}
    
    for bin_i in range(len(dset_bins)-1):
        result_dict[bin_i] = {}
        indices = ((dset > dset_bins[bin_i]) &
                   (dset < dset_bins[bin_i+1]))

        # initalize the outputs
        result_dict[bin_i]['50_phys'] = np.ma.median(ys[indices,:], axis=0)
        result_dict[bin_i]['16_phys'] = np.percentile(ys[indices,:], 16, axis=0)
        result_dict[bin_i]['84_phys'] = np.percentile(ys[indices,:], 84, axis=0)
        result_dict[bin_i]['mean_phys'] = np.ma.mean(ys[indices,:], axis=0)
        result_dict[bin_i]['std_phys'] = np.ma.std(ys[indices,:], axis=0)

        result_dict[bin_i]['Ngal'] = len(indices[indices])
        
    # finish loop over bins. return to main
    return result_dict


def plot_stacked_dict(stacked_dict, dset, dset_savelabel, dset_bins, dset_label, colors, radii='phys', savefig=False):
    """
    Plot the stacked_dict given the dict, dset_bins, and label.
    If radii=='norm', then use the normalized radial coordinates.
    If radii=='phys', use physical radial coordinates.
    """
    if radii == 'phys':
        x_label = r'Satellite-Centric Distance [kpc]'
        x = xs[0]
    else:
        raise ValueError('invalid radii value %s. Accepted values are norm and phys'%radii)
        
    if len(colors) < len(stacked_dict.keys()):
        raise ValueError('too few colors given')
    
    fig, ax = plt.subplots()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(x_label)
    ax.set_ylabel(LXsoftexcess_label)

    for i, bin_key in enumerate(stacked_dict.keys()):
        color = colors[i]
        y_50 = stacked_dict[bin_key]['50_%s'%radii]
        y_16 = stacked_dict[bin_key]['16_%s'%radii]
        y_84 = stacked_dict[bin_key]['84_%s'%radii]
        
        y_mean = stacked_dict[bin_key]['mean_%s'%radii]
        y_std = stacked_dict[bin_key]['std_%s'%radii]
        
        Ngal = stacked_dict[bin_key]['Ngal']
        ax.plot(x, y_50, '--', color=color, lw=2)
        ax.fill_between(x, y_16, y_84, color=color, alpha=0.1)
        
        ax.plot(x, y_mean, '-', color=color, label=r'$[%.1f, %.1f)\ (%d)$'%(dset_bins[i], dset_bins[i+1], Ngal), lw=4)
        
    ax.set_ylim(9e-2, 2e1)
    ax.legend(title=dset_label)
    ax.set_title(r'TNG-Cluster $z=0$ halo %d satellites'%haloID)

    
    if (savefig):
        direc = '../Figures/TNGCluster/'
        fname = '%s_snap099_haloID%08d_satllite-xray-SB-excess_%s_stack.pdf'%(sim, haloID, dset_savelabel)
        fig.savefig(direc + fname, bbox_inches='tight')
    
    return fig, ax


In [ ]:
savefig=False

subhalos_mgas_plot = subhalos_mgas.copy()
subhalos_mgas_plot[subhalos_mgas == 0] = 1.1e6
subhalos_mgas_plot = np.log10(subhalos_mgas_plot)
mgas_bins = [6.0, 8.0, 10.0, 13.0]
stacked_dict_mgas = return_stacked_dict(ys, subhalos_mgas_plot, mgas_bins)
colors = ['salmon', 'red', 'darkred']
savelabel = 'mgas'
fig, ax = plot_stacked_dict(stacked_dict_mgas, subhalos_mgas_plot, savelabel, mgas_bins, r'$\log_{10}[M_{\rm gas}^{\rm sat} / {\rm M_\odot}] \in$', colors, radii='phys', savefig=savefig)


subhalos_mstar_plot = np.log10(subhalos_mstar)
mstar_bins = [10.0, 10.5, 11.0, 12.0]
stacked_dict_mstar = return_stacked_dict(ys, subhalos_mstar_plot, mstar_bins)
colors = ['lightblue', 'blue', 'darkblue']
savelabel = 'mstar'
fig, ax = plot_stacked_dict(stacked_dict_mstar, subhalos_mstar_plot, savelabel, mstar_bins, r'$\log_{10}[M_{\star}^{\rm sat} / {\rm M_\odot}] \in$', colors, radii='phys', savefig=savefig)


subhalos_hcd_plot = subhalos_hcd_norm.copy()
hcd_bins = [0.0, 0.5, 1.0, 1.501]
stacked_dict_hcd = return_stacked_dict(ys, subhalos_hcd_plot, hcd_bins)
colors = ['violet', 'mediumpurple', 'indigo']
savelabel = 'hcd'
fig, ax = plot_stacked_dict(stacked_dict_hcd, subhalos_hcd_plot, savelabel, hcd_bins, r'proj. $d_{\rm sat}^{\rm host} / R_{\rm 200c}^{\rm host} \in$', colors, radii='phys', savefig=savefig)


In [ ]:

fig, ax = plt.subplots()
ys_median = np.ma.median(ys, axis=0)
ys_16s = np.percentile(ys, 16, axis=0)
ys_84s = np.percentile(ys, 84, axis=0)
ys_mean = np.ma.mean(ys, axis=0)
ax.plot(xs.T, ys.T, 'k-', alpha=0.5, lw=0.5)
ax.plot(x * pixel_size, ys_median, 'b--', lw=4)
ax.plot(x * pixel_size, ys_mean, 'r--', lw=4)
ax.fill_between(x * pixel_size, ys_16s, ys_84s, color='tab:gray', alpha=1.0)
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
group_index = 33
haloID = haloIDs[group_index]
satellite_taudict_indices = np.where(haloID == HostSubhaloGrNr)[0]
subhaloIDs = subfindIDs[satellite_taudict_indices]
group = Lx_maps[central_keys[group_index]]

proj = 'xy'
FoF_Lxsoft = group['xray_lum_0.5-2.0kev_%s'%proj]
FoF_Lxsoft_smooth = group['xray_lum_0.5-2.0kev_%s_smooth'%proj]

subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
for i, taudict_i in enumerate(satellite_taudict_indices):
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_axes = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_axes[:,0]) < R200c*1.5) & (abs(subhaloPos_axes[:,1]) < R200c*1.5))[0]

print(satellite_taudict_indices.size, satellites_fov_indices.size)

size = 3.0e2 # kpc
x = np.logspace(np.log10(1./2), np.log10(size / pixel_size))
xs = np.tile(x, (satellites_fov_indices.size,1)) * pixel_size
ys = np.zeros((satellites_fov_indices.size, x.size), dtype=x.dtype) - 1
for i, satellite_fov_index in enumerate(satellites_fov_indices):
    subhaloID = subhaloIDs[satellite_fov_index]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
    SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)


    center = [(FoF_LXsoftexcess.shape[0] - 1.) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
    extent_indices = [int(center[0] + (SubhaloPos[axes][1] - size)/pixel_size), int(center[0] + (SubhaloPos[axes][1] + size)/pixel_size),
                      int(center[1] + (SubhaloPos[axes][0] - size)/pixel_size), int(center[1] + (SubhaloPos[axes][0] + size)/pixel_size)]
    if min(extent_indices) < 0:
        print('satellite on the edge of the FoV: i %d'%i)
        continue
    vals = FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]]
    model, func = model_2dradprof(vals)
    ys[i,:] = func(x)
    
ys = np.ma.masked_values(ys, -1)

In [ ]:
group_index = 33
haloID = haloIDs[group_index]
satellite_taudict_indices = np.where(haloID == HostSubhaloGrNr)[0]
subhaloIDs = subfindIDs[satellite_taudict_indices]
group = Lx_maps[central_keys[group_index]]

proj = 'xy'
FoF_Lxsoft = group['xray_lum_0.5-2.0kev_%s'%proj]
FoF_Lxsoft_smooth = group['xray_lum_0.5-2.0kev_%s_smooth'%proj]

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
R200c = halo['Group_R_Crit200'] * a / h
halo_pos = halo['GroupPos'] * a / h

# we start by ignoring the z axis and a pixel scale of 1kpc

axes = [0,1]
boxSizeImg = [3.*R200c, 3.*R200c] # kpc
boxCen = halo_pos[axes]

# now plot data, model, zoom-in and rad prof
fig, big_axs = plt.subplots(2, 3, figsize=(figsizewidth, figsizewidth/1.4))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness [0.2-2.0 keV]'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
extent = [-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.]
vmax = np.percentile(FoF_Lxsoft, 99.9)
Lx_imshow_kwargs = dict(extent=extent, cmap=cmap, norm=mpl.colors.LogNorm(vmin=1.0e4, vmax=vmax), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'[$10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$]'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

### top row
axs = big_axs[0,:]

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(FoF_Lxsoft, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add a circle at R200c
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)

# add a length scale
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All Gas', fontsize='small')


ax = axs[1]
img = ax.imshow(FoF_Lxsoft_smooth, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Smoothed Model [%d kpc]'%(int(smoothing_scale * pixel_size)), fontsize='small')

# plot the radial profile
FoF_Lxsoft_RP, FoF_Lxsoft_RP_func = model_2dradprof(FoF_Lxsoft)
FoF_Lxsoft_smooth_RP, FoF_Lxsoft_smooth_RP_func = model_2dradprof(FoF_Lxsoft_smooth)
ax = axs[2]
x = np.logspace(np.log10(1./2.), np.log10(boxSizeImg[0]/2. / pixel_size))
ax.plot(x * pixel_size, FoF_Lxsoft_RP_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance [kpc]', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

### bottom row
axs = big_axs[1,:]

FoF_LXsoftexcess = (FoF_Lxsoft / FoF_Lxsoft_smooth)

vmax = np.percentile(FoF_LXsoftexcess, 99.9)

ax = axs[0]
norm = mpl.colors.LogNorm(vmin=10.**(-np.log10(vmax)), vmax=vmax)
excess_kwargs = dict(cmap='Spectral', origin='lower', norm=norm)
img = ax.imshow(FoF_LXsoftexcess, extent=extent, **excess_kwargs)
cbar = fig.colorbar(img, ax=axs[1], extend='both')
LXsoftexcess_label = 'Soft X-ray SB Excess \n [All Gas / Model]'
cbar.set_label(LXsoftexcess_label, fontsize='small')

# add the R200c circle
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', color='black', fill=False)
ax.add_patch(circ_R200c)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(LXsoftexcess_label, fontsize='small')

# add a rectangle around the subhalo
satellite_index = 13
subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
for i, taudict_i in enumerate(satellite_taudict_indices):
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

SubhaloPos_xy = subhaloPos[:,axes]
satellites_fov_indices = np.where(np.sqrt(SubhaloPos_xy[:,axes[0]]**2 + SubhaloPos_xy[:,axes[1]]**2) < R200c)[0]

satellite_fov_index = satellites_fov_indices[satellite_index]
subhaloID = subhaloIDs[satellite_fov_index]
subhalopos_xy = SubhaloPos_xy[satellite_fov_index]

satellite_size = 3.0e2 # kpc

# draw a square around the satellite, and include the zoom lines
xy = (subhalopos_xy[0] - satellite_size, subhalopos_xy[1] - satellite_size)
height = satellite_size*2
width = satellite_size*2

rect = patches.Rectangle(xy, width, height, fill=False)
ax.add_patch(rect)

# add the zoom lines 

ax1 = axs[0]
ax2 = axs[1]
con = ConnectionPatch(xyA=xy, coordsA=ax1.transData,
                      xyB=(0,0), coordsB=ax2.transAxes)
fig.add_artist(con)

xy2 = (xy[0], xy[1] + height)
con = ConnectionPatch(xyA=xy2, coordsA=ax1.transData,
                      xyB=(0,1), coordsB=ax2.transAxes)
fig.add_artist(con)

ax = axs[1]
center = [(FoF_LXsoftexcess.shape[0] - 1) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
extent_indices = [int(round(center[0] + (subhalopos_xy[1] - satellite_size)/pixel_size)),
                  int(round(center[0] + (subhalopos_xy[1] + satellite_size)/pixel_size)),
                  int(round(center[1] + (subhalopos_xy[0] - satellite_size)/pixel_size)),
                  int(round(center[1] + (subhalopos_xy[0] + satellite_size)/pixel_size))]
extent_satellite = [-satellite_size, satellite_size, -satellite_size, satellite_size]

vals = FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]]
img = ax.imshow(vals, extent=extent_satellite, **excess_kwargs)

x0 = - satellite_size + (satellite_size*0.15) # kpc
y0 = satellite_size - (satellite_size*0.1) # kpc
length = 1.0e2 # kpc
ax.plot([x0, x0+length], [y0, y0], color='black', marker='None', ls='-')
ax.text(x0+length/2., y0-(satellite_size * 0.1), '%d kpc'%(length), ha='center', va='top', fontsize='small', color='black')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Zoom-in on an \n example satellite (%d)'%subhaloID, fontsize='small')
levels = [1., 2., 5., 10.]
colors = ['lightgray', 'darkgray', 'tab:gray', 'black']

vals_fft = np.fft.fft2(vals)
vals_smooth_fft = fourier_gaussian(vals_fft, contours_smoothing)
vals_smoothed = np.fft.ifft2(vals_smooth_fft).real
vals_smoothed = gaussian_filter(vals, contours_smoothing)
ax.contour(vals_smoothed, levels=[1, 2., 5., 10.], colors=colors, extent=extent_satellite)

ax = axs[2]
model, func = model_2dradprof(vals_smoothed)
x = np.logspace(np.log10(1.), np.log10(satellite_size / pixel_size))
ax.plot(x * pixel_size, func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Satellite-Centric Distance [kpc]', fontsize='small')
intercepts = np.interp(levels, func(np.arange(satellite_size / pixel_size))[::-1], np.arange(satellite_size / pixel_size)[::-1]) * pixel_size
ax.hlines(levels, 0, intercepts, colors=colors, ls='--')
ax.vlines(intercepts, 0, levels, colors=colors, ls='--')

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_subahlo%08d_%s_multipanel.pdf'%(sim, haloID, subhaloID, proj)
    fig.savefig(direc + fname, bbox_inches='tight', dpi=600)


In [ ]:
fig, ax = plt.subplots()
ys_median = np.ma.median(ys, axis=0)
ys_mean = np.ma.mean(ys, axis=0)
ax.plot(xs.T, ys.T, 'k-', lw=1)
ax.plot(x * pixel_size, ys_median, 'b--', lw=4)
ax.plot(x * pixel_size, ys_mean, 'r--', lw=4)
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
import numpy as np
from astropy import units as u
import sys
h_const = 0.673 # Hubble constant
k_B_cgs = 1.38064852e-16 # boltzmann constant (CGS)
m_p_cgs = 1.6726219e-24 # proton mass [gram]
X_H = 0.76 # hydrogen mass fraction
gamma_gas = 5.0/3


def calculate_Xray_BolometricLum_Arepo(data):
    """ This function (Written by D. Nelson with minor modifications by M. Ayromlou) computes X-ray luminosity using Navarro+ (1995) Eqn. 6, the most basic estimator of bolometric X-ray luminosity in [10^30 erg/s] for individual gas cells, based only on their density and temperature. It assumes simplified (primordial) high-temp cooling function, and only free-free (bremsstrahlung) emission contribution from T>10^6 Kelvin gas. All inputs in code (Arepo) units. """

    # calculate mean molecular weight
    mu = calculate_mean_molecular_weight_Arepo(data).astype('float64')

    # calculate temperature (keV)
    temp = calculate_temperature_Arepo(data,unit='Kelvin')
    temp_Kelvin = temp * u.Kelvin
    temp_keV = temp_Kelvin.to(u.keV, equivalencies=u.temperature_energy()) / u.keV 

    # Eqn. 6
    mass_cgs = convert_units_Arepo(data['Masses'],quantity='mass',MAorCGS='CGS')
    rho_cgs = convert_units_Arepo(data['Density'],quantity='density',MAorCGS='CGS')

    L_x = (1.2e-24/ (mu**2)) * mass_cgs *  rho_cgs * np.sqrt(temp_keV)

    # clip any cells on eEOS (SFR>0) to zero
    dummy = np.where(data['StarFormationRate'] > 0.0)
    L_x[dummy] = 0.0

    # implement a linear ramp from log(T)=6.0 to log(T)=5.8 over which we clip to zero. This step is necessary to avoid having a non-zero X-ray luminosity for gas with T<10^6 K, which is not expected from Navarro+95 Eqn. 6.
    temp = np.log10(temp)
    L_x *= np.clip( (temp-5.8)/0.2, 0.0, 1.0 )

    L_x = np.array(L_x/1e30,dtype=np.float32) # work in this unit system of [10^30 erg/s] for xray to avoid overflows to inf

    return (L_x)


def calculate_mean_molecular_weight_Arepo(data):
    # mean molecular weight of the gas: see TNG FAQ webpage for details
    e_abund = data['ElectronAbundance']
    mu = 4*m_p_cgs/(1 + 3*X_H + 4*X_H*e_abund) # mean molecular weight
    return(mu)

def calculate_temperature_Arepo(data,unit='Kelvin'):
    # Temperature of the gas: see TNG FAQ webpage for details
    i_energy = data['InternalEnergy']
    mu = calculate_mean_molecular_weight_Arepo(data)
    gas_temp = (gamma_gas-1)*(i_energy/k_B_cgs)*mu*(10**10) # kelvin
    if unit != 'Kelvin':
        gas_temp = gas_temp * u.Kelvin
        if unit == 'keV':
            gas_temp = gas_temp.to(u.keV, equivalencies=u.temperature_energy()) / u.keV 
        elif unit == 'eV':
            gas_temp = gas_temp.to(u.eV, equivalencies=u.temperature_energy()) / u.eV
    return(gas_temp)


def convert_units_Arepo(data, quantity, tofromcode='fromcode', MAorCGS='MA'):
    """ This function converts units from Arepo units to my prefered units 
    * data: data
    * quantity [Arepo unit][my preferred unit]: mass [1e10Msun/h][Msun], scale[kpc/h][Mpc], density, density2d
    * tofromcode: tocode (convert from my units to code units), fromcode (convert from code units to my units)
    """
    
    if MAorCGS == 'MA':
        mass_conversion = 1e10/h_const
        scale_conversion = 1/(1000*h_const)
    elif MAorCGS == 'CGS':
        mass_conversion = (1e10/h_const)*1.989e33 
        scale_conversion = (3.086e24)/(1000*h_const)
    
    if quantity == 'mass':
        if tofromcode == 'fromcode':
            data = data*mass_conversion
        elif tofromcode == 'tocode':
            data = data/mass_conversion
    elif quantity == 'scale':
        if tofromcode == 'fromcode':
            data = data*scale_conversion
        elif tofromcode == 'tocode':
            data = data/scale_conversion
    elif quantity == 'density':
        if tofromcode == 'fromcode':
            data = data*mass_conversion/(scale_conversion**3)
        elif tofromcode == 'tocode':
            data = data*(scale_conversion**3)/mass_conversion
    elif quantity == 'density2d':
        if tofromcode == 'fromcode':
            data = data*mass_conversion/(scale_conversion**2)
        elif tofromcode == 'tocode':
            data = data*(scale_conversion**2)/mass_conversion
    else:
        print("Quantity not recognized")
        sys.exit()
            
    return(data)

In [ ]:
gas_fields = ['Masses', 'Coordinates', 'Density', 'GFM_Metallicity',
              'InternalEnergy', 'ElectronAbundance', 'StarFormationRate']

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
halo_gas_cells = il.snapshot.loadHalo(basePath, snapNum, haloID, gas_ptn, fields=gas_fields)
halo_gas_cells = ru.calc_temp_dict(halo_gas_cells)

In [ ]:
Lx_MA = calculate_Xray_BolometricLum_Arepo(halo_gas_cells)

In [ ]:
indices = Lx_MA > 0
plt.figure()
plt.hist(np.log10(Lx_MA[indices]) + 30)
plt.yscale('log')


In [ ]:
subset = il.snapshot.getSnapOffsets(basePath, snapNum, haloID, "Group")
start = subset['offsetType'][gas_ptn]
end = start +  subset['lenType'][gas_ptn]



In [ ]:
subset = il.snapshot.getSnapOffsets(basePath, snapNum, haloID, "Group")
start = subset['offsetType'][gas_ptn]
end = start +  subset['lenType'][gas_ptn]

direc = '/vera/ptmp/gc/dnelson/sims.TNG/L680n8192TNG/data.files/cache/'
fname = 'cached_gas_xray_lum_0.5-2.0kev_99.hdf5'
with h5py.File(direc + fname, 'r') as f:
    Lx_DN = f['xray_lum_0.5-2.0kev'][start:end]
    f.close()

In [ ]:
indices = Lx_DN > 0
plt.figure()
plt.hist(np.log10(Lx_DN[indices]) + 30)
plt.yscale('log')

In [ ]:
indices = Lx_MA > 0
y = Lx_MA[indices] / Lx_DN[indices]
x = Lx_DN[indices]
T = halo_gas_cells['Temperature'][indices]
rho = halo_gas_cells['Density'][indices] * 1.0e10 / h / (a / h)**3
Z = halo_gas_cells['GFM_Metallicity'][indices] / 0.0127

direc = '../Figures/TNGCluster/'
savefig=False

fig, axs = plt.subplots(2, 2, figsize=(figsizewidth, figsizewidth / 1.25))

xdsets = [np.log10(x) + 30, np.log10(T), np.log10(rho), np.log10(Z)]
xlabels = [r'$\log_{10}\, [L_{\rm 0.2-2\, keV}({\rm APEC}) / {\rm erg\, s^{-1}}]$',
           r'$\log_{10}\, [T_{\rm gas} / {\rm K}]$',
           r'$\log_{10}\, [\rho_{\rm gas} / {\rm M_\odot\, kpc^{-3}}]$',
           r'$\log_{10}\, [Z_{\rm gas} / {\rm Z_\odot}]$']

fig.suptitle(r'TNG-Cluster snap 099 halo %d gas ($%s$ cells)'%(haloID, ru.latex_float(x.size, 1)))
fig.supylabel(r'$\log_{10}\, [L_{\rm x}({\rm Navarro+95}) /L_{\rm 0.2-2\, keV}({\rm APEC})]$')

one_one_kwargs = dict(ls='-', marker='None', c='k', zorder=1.0, lw=2.5)

norm = mpl.colors.LogNorm(vmax=1.0e5)
bins = 500
Ncells_label = r'Number of Gas Cells'
for i, ax in enumerate(axs.flatten()):
    dset = xdsets[i]
    xlabel = xlabels[i]
    ax.set_xlabel(xlabel)

    vals = ax.hist2d(dset, np.log10(y), bins=bins, norm=norm, rasterized=True)
    ax.plot(ax.get_xlim(), [0, 0], **one_one_kwargs)

    
    cax = inset_axes(ax, width='50%', height='5%', loc='lower left')
    cbar = plt.colorbar(vals[3], cax=cax, orientation='horizontal')
    cbar.set_label(Ncells_label, labelpad=5, fontsize='small')
    cbar.ax.tick_params(labelsize='x-small')
    cax.xaxis.set_label_position('top')
    cax.xaxis.set_ticks_position('top')

if savefig:
    fname = '%s_snap099_haloID%08d_LxNavarro+95-LxAPEC_multipanel.pdf'%(sim, haloID)
    fig.savefig(direc + fname, bbox_inches='tight')

    